# Rule-based system

In [ ]:
# Import pandas and load sentences file
import pandas as pd

# Choose which file to predict
df = pd.read_csv('Dataset_sentences.csv')

In [ ]:
# Import regex and define the regex function
import re
Keywords = ['data\s*(?:set|base)s?', 'corpus', 'corpora', 'tree\s*bank', 
            '(?:train|test|validation|testing|trainings?)\s*(?:set|data)',
            'collections?', 'benchmarks?']

expression = re.compile(r'\b(' + '|'.join(Keywords) + r')\b', flags = re.IGNORECASE)

In [ ]:
# Import spaCy and load in the matcher
import spacy
from spacy.matcher import Matcher
nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)

In [ ]:
# Import NLTK for the POS-tags
import nltk

In [ ]:
# Initialise dataframe to put the predictions in 
dataset = pd.DataFrame(columns= ['Sentence #', 'Word', 'POS', 'Tag'])
empty = []
x=0

# Loop through all sentences
for _,row in df.iterrows():
    sentence = row.text
    number = row.id
    matcher = Matcher(nlp.vocab)
    match = re.findall(expression, sentence)
    store = {}
    store2 = {}
    
    
    # When the sentence contains a keyword it gets put into the matcher
    if len(match) > 0:
        doc = nlp(sentence)
        
        # pattern one or two get the .... dataset for example
        pattern = [{"IS_PUNCT": False, 'POS':{'IN': ['PROPN', 'NOUN', 'NUM']}, 'OP':'?'},{'LENGTH':{'>=': 2},'LOWER': {'NOT_IN': ['table', 'figure', 'results','data']},'IS_LOWER': False, "IS_PUNCT": False, 'IS_DIGIT':False, 'POS':{'IN': ['PROPN', 'NOUN', 'ADJ']}},
                {'LOWER':{'NOT_IN': [':', '.','datasets', 'data', 'databases']},'POS': {'IN': ['PROPN', 'NOUN', 'NUM', 'PUNCT', 'ADJ']}, 'OP':'*'},
                {'TEXT': {'IN': ['and', 'or']}, 'OP':'?'},
                {'LOWER':{'NOT_IN': [':', '.','datasets', 'data', 'databases']},'POS': {'IN': ['PROPN', 'NOUN', 'NUM', 'PUNCT', 'ADJ']}, 'OP':'*'},
                {"TEXT": {"REGEX": "((:?d|D)ata\s*(?:set|base)s?$|(:?C|c)orpus$|(:?C|c)orpora$|(:?T|t)ree\s*bank$|(:?C|c)ollections?$|(:?B|b)enchmarks?)$|trainval$"}}]
        pattern2 = [{"IS_PUNCT": False, 'POS':{'IN': ['PROPN', 'NOUN', 'NUM']}, 'OP':'?'},{'LENGTH':{'>=': 2},'LOWER': {'NOT_IN': ['table', 'figure', 'results','data']},'IS_LOWER': False, "IS_PUNCT": False, 'IS_DIGIT':False, 'POS':{'IN': ['PROPN', 'NOUN', 'ADJ']}},
                {'LOWER':{'NOT_IN': [':', '.','datasets', 'data', 'databases']},'POS': {'IN': ['PROPN', 'NOUN', 'NUM', 'PUNCT', 'ADJ']}, 'OP':'*'},
                {'TEXT': {'IN': ['and', 'or']}, 'OP':'?'},
                {'LOWER':{'NOT_IN': [':', '.','datasets', 'data', 'databases']},'POS': {'IN': ['PROPN', 'NOUN', 'NUM', 'PUNCT', 'ADJ']}, 'OP':'*'},
                {"TEXT": {"REGEX": "(train$|test$|validation$|testing$|trainings?$|data$)"}},
                {"TEXT": {"REGEX": "(sets?|(:?D|d)atasets?|(:?D|d)atabases?)"}}]
        matcher.add('Dataset', None, pattern)
        matcher.add('Dataset', None, pattern2)
        matches = matcher(doc)
        
        # picks up things with a wider range 
        matcher = Matcher(nlp.vocab) 
        pattern3 = [{"IS_PUNCT": False, 'POS':{'IN': ['PROPN', 'NOUN', 'NUM']}, 'OP':'?'},{'LENGTH':{'>=': 2},'LOWER': {'NOT_IN': ['table', 'figure', 'results','data', 'dataset', 'database', 'references', 'discussion']},'IS_LOWER': False, "IS_PUNCT": False, 'IS_DIGIT':False, 'POS':{'IN': ['PROPN', 'NOUN']}},
                   {'LOWER':{'NOT_IN': [':', '.','datasets', 'data', 'databases']},'POS': {'IN': ['PROPN', 'NOUN', 'NUM', 'PUNCT']}, 'OP':'*'},
                    {'TEXT': 'is'}, {'TEXT': 'a'}, {'LOWER':{'NOT_IN': [':', '.','datasets', 'data', 'databases']},'POS': {'IN': ['PROPN', 'NOUN', 'NUM', 'PUNCT']}, 'OP':'*'},
                    {"TEXT": {"REGEX": "((:?d|D)ata\s*(?:set|base)s?$|(:?C|c)orpus$|(:?C|c)orpora$|(:?T|t)ree\s*bank$|(:?C|c)ollections?$|(:?B|b)enchmarks?)$|trainval$"}}]
        pattern4 = [{"IS_PUNCT": False, 'POS':{'IN': ['PROPN', 'NOUN', 'NUM']}, 'OP':'?'},{'LENGTH':{'>=': 2},'LOWER': {'NOT_IN': ['table', 'figure', 'results','data', 'dataset', 'database', 'references', 'discussion']},'IS_LOWER': False, "IS_PUNCT": False, 'IS_DIGIT':False, 'POS':{'IN': ['PROPN', 'NOUN']}},
                   {'LOWER':{'NOT_IN': [':', '.','datasets', 'data', 'databases']},'POS': {'IN': ['PROPN', 'NOUN', 'NUM', 'PUNCT']}, 'OP':'*'},
                    {'TEXT': 'is'}, {'TEXT': 'a'}, {'LOWER':{'NOT_IN': [':', '.','datasets', 'data', 'databases']},'POS': {'IN': ['PROPN', 'NOUN', 'NUM', 'PUNCT']}, 'OP':'*'},
                    {"TEXT": {"REGEX": "(train$|test$|validation$|testing$|trainings?$|data$)"}},
                    {"TEXT": {"REGEX": "(sets?|(:?D|d)atasets?|(:?D|d)atabases?)"}}]
        
        # picks op the .... dataset
        pattern5 = [{'LOWER':'the'},{'POS':{'IN': ['PROPN']}, 'LOWER': {'NOT_IN': ['training','test','testing']},'IS_LOWER' : True},
                    {"TEXT": {"REGEX": "((:?d|D)ata\s*(?:set|base)s?$|(:?C|c)orpus$|(:?C|c)orpora$|(:?T|t)ree\s*bank$|(:?C|c)ollections?$|(:?B|b)enchmarks?)$|trainval$"}}]
        pattern6 =  [{'LOWER':'the'},{'POS':{'IN': ['PROPN']}, 'LOWER': {'NOT_IN': ['training','test','testing']}, 'IS_LOWER' : True},
                     {"TEXT": {"REGEX": "(train$|test$|validation$|testing$|trainings?$|data$)"}},
                    {"TEXT": {"REGEX": "(sets?|(:?D|d)atasets?|(:?D|d)atabases?)"}}]
        matcher.add('Dataset', None, pattern3)
        matcher.add('Dataset', None, pattern4)
        matcher.add('Dataset', None, pattern5)
        matcher.add('Dataset', None, pattern6)
        matches2 = matcher(doc)
        for match_id, start, end in matches:
            try: 
                store[end]
            except:
                store[end] = start
            if store[end] > start:
                store[end] = start
        for match_id, start, end in matches2:
            if str(doc[start]).lower() == 'the':
                try: 
                    store[end]
                except:
                    store[end] = start + 1
                if store[end] > start + 1:
                    store[end] = start + 1
            else:
                test = str(doc[start:end])
                test2 = re.split(' \[\d+] is a ', test)
                test3 = test.split(' is a ')
                if len(test2) == 2:
                    end = start + len(test2[0].split(' '))
                else:
                    end = start + len(test3[0].split(' '))
                try: 
                    store[end]
                except:
                    store[end] = start
                if store[end] > start:
                    store[end] = start
        if ':' in str(doc[0:5]):
            matcher = Matcher(nlp.vocab)
            
            # picks up sentences like Imagenet: ... 
            pattern8 = [{'LENGTH':{'>=': 2},'LOWER': {'NOT_IN': ['left','table', 'figure','results', 'discussion', 'datasets', 'database', 'dataset', 'databases', 'experiments', 'object', 'benchmark', 'experimental', 'problem', 'input', 'detection', 'data', 'classification', 'lite', 'training', 'tuning', 'experiment', 'top', 'next', 'trajectories', 'baselines', 'baseline', 'output', 'corpus','inputs', 'images', 'digits', 'ppos', 'topic', 'program']},'IS_LOWER': False, "IS_PUNCT": False, 'IS_DIGIT':False, 'POS':{'IN': ['PROPN', 'NOUN']}}, 
                        {'LOWER': {'NOT_IN':['.', ')', ':', 'table', 'figure','results', 'cover', 'alogirthm','histograms']},'IS_LOWER':False, 'POS':{'IN': ['PROPN', 'NOUN', 'NUM', 'ADJ']}, 'OP':'*'},{'TEXT': ':'}]
            matcher.add('Dataset', None, pattern8)
            doc2 = ' '.join([str(token) for token in doc[0:5]])
            doc3 = nlp(doc2)
            matches = matcher(doc3)
            for match_id, start, end in matches:
                try: 
                    store[end-1]
                except:
                    store[end-1] = start 
                if store[end-1] > start :
                    store[end-1] = start
                
        # deletes picks with a in front of it        
        for key in list(store):
            start = store[key]
            if str(doc[start-1]).lower() in ['a', 'an']:
                del store[key]
                continue
            elif str(doc[start-2]).lower() in ['a', 'an']:
                del store[key]
                continue
            elif str(doc[start-3]).lower() in ['a', 'an']:
                del store[key]
                continue
        for key in store.keys():
            try:
                store2[store[key]]
            except:
                store2[store[key]] = key
            if store2[store[key]] < key:
                store2[store[key]] = key
            
        if len(store) == 0:
            matcher = Matcher(nlp.vocab)
            
            # picks up examples like on MNIST
            pattern7 = [{'TEXT':'on'},{'IS_LOWER': False, "IS_PUNCT": False, 'IS_DIGIT':False, 'POS':{'IN': ['PROPN', 'NOUN']}, 'LENGTH':{'>=': 2}}, 
                        {'TEXT': {'NOT_IN':['.', ')']},'IS_LOWER':False, 'OP':'*'}]
            matcher.add('Dataset', None, pattern7)
            matches = matcher(doc) 
            for match_id, start, end in matches:
                try: 
                    store[end]
                except:
                    store[end] = start + 1
                if store[end] > start + 1:
                    store[end] = start + 1
            for key in store.keys():
                try:
                    store2[store[key]]
                except:
                    store2[store[key]] = key
                if store2[store[key]] < key:
                    store2[store[key]] = key
                
                
        if len(store) == 0:
            matcher = Matcher(nlp.vocab)
            if 'datasets' in sentence or 'databases' in sentence or 'collections' in sentence or 'corpora' in sentence or 'benchmarks' in sentence or 'Datasets' in sentence or 'Databases' in sentence or 'Collections' in sentence or 'Corpora' in sentence or 'Benchmarks' in sentence:
                matcher = Matcher(nlp.vocab)
                
                # picks op things like the following datasets: MNIST, SVHN & Imagenet
                pattern9 = [{'LOWER': {'IN': ['datasets', 'collections', 'corpora', 'databases', 'benchmarks', '|']}},{'TEXT':{'IN':[',',':']}, 'OP':'?'},{'TEXT':'like', 'OP':'?'}, 
                           {'LENGTH':{'>=': 2},'IS_LOWER': False, "IS_PUNCT": False, 'IS_DIGIT':False, 'POS':{'IN': ['PROPN', 'NOUN']}}, 
                           {'IS_LOWER': False, 'OP':'*'}, {'TEXT':',', 'OP':'?'},
                          {'IS_LOWER': False, "IS_PUNCT": False, 'IS_DIGIT':False, 'POS':{'IN': ['PROPN', 'NOUN']}, 'OP':'*'}, 
                           {'IS_LOWER': False, 'OP':'*'}, {'TEXT':',', 'OP':'?'},
                          {'IS_LOWER': False, "IS_PUNCT": False, 'IS_DIGIT':False, 'POS':{'IN': ['PROPN', 'NOUN']}, 'OP':'*'}, 
                           {'IS_LOWER': False, 'OP':'*'}, {'TEXT':',', 'OP':'?'},
                          {'IS_LOWER': False, "IS_PUNCT": False, 'IS_DIGIT':False, 'POS':{'IN': ['PROPN', 'NOUN']}, 'OP':'*'}, 
                           {'IS_LOWER': False, 'OP':'*'}, {'TEXT':',', 'OP':'?'},
                          {'IS_LOWER': False, "IS_PUNCT": False, 'IS_DIGIT':False, 'POS':{'IN': ['PROPN', 'NOUN']}, 'OP':'*'}, 
                           {'IS_LOWER': False, 'OP':'*'}, {'TEXT':'and', 'OP':'?'},
                          {'IS_LOWER': False, "IS_PUNCT": False, 'IS_DIGIT':False, 'POS':{'IN': ['PROPN', 'NOUN']}, 'OP':'*'}, 
                           {'IS_LOWER': False, 'OP':'*'},{'IS_LOWER':False, 'OP':'?'}]
                pattern10 = [{'LOWER': {'IN': ['datasets', 'collections', 'corpora', 'databases', 'benchmarks', '|']}},{'TEXT':'for'},{'OP':'*'},{'TEXT':{'IN':[',',':']}, 'OP':'?'}, 
                           {'LENGTH':{'>=': 2},'IS_LOWER': False, "IS_PUNCT": False, 'IS_DIGIT':False, 'POS':{'IN': ['PROPN', 'NOUN']}}, 
                           {'IS_LOWER': False, 'OP':'*'}, {'TEXT':',', 'OP':'?'},
                          {'IS_LOWER': False, "IS_PUNCT": False, 'IS_DIGIT':False, 'POS':{'IN': ['PROPN', 'NOUN']}, 'OP':'*'}, 
                           {'IS_LOWER': False, 'OP':'*'}, {'TEXT':',', 'OP':'?'},
                          {'IS_LOWER': False, "IS_PUNCT": False, 'IS_DIGIT':False, 'POS':{'IN': ['PROPN', 'NOUN']}, 'OP':'*'}, 
                           {'IS_LOWER': False, 'OP':'*'}, {'TEXT':',', 'OP':'?'},
                          {'IS_LOWER': False, "IS_PUNCT": False, 'IS_DIGIT':False, 'POS':{'IN': ['PROPN', 'NOUN']}, 'OP':'*'}, 
                           {'IS_LOWER': False, 'OP':'*'}, {'TEXT':',', 'OP':'?'},
                          {'IS_LOWER': False, "IS_PUNCT": False, 'IS_DIGIT':False, 'POS':{'IN': ['PROPN', 'NOUN']}, 'OP':'*'}, 
                           {'IS_LOWER': False, 'OP':'*'}, {'TEXT':'and', 'OP':'?'},
                          {'IS_LOWER': False, "IS_PUNCT": False, 'IS_DIGIT':False, 'POS':{'IN': ['PROPN', 'NOUN']}, 'OP':'*'}, 
                           {'IS_LOWER': False, 'OP':'*'},{'IS_LOWER':False, 'OP':'?'}]
                matcher.add('Dataset', None, pattern9)
                matcher.add('Dataset', None, pattern10)
                matches = matcher(doc)
                for match_id, start, end in matches:
                    try: 
                        store[end]
                    except:
                        store[end] = start
                    if store[end] > start:
                        store[end] = start
                for key in store.keys():
                    try:
                        store2[store[key]]
                    except:
                        store2[store[key]] = key
                    if store2[store[key]] < key:
                        store2[store[key]] = key
                
                multiples = []
                for key in store2.keys():
                    multiples.append([key,store2[key]])
                for multiple in multiples:
                    store = {}
                    store2 = {}
                    matcher = Matcher(nlp.vocab)
                    
                    # Removes the keywords at the matches before
                    pattern11 = [{'LENGTH':{'>=': 2},'LOWER': {'NOT_IN': ['datasets', 'collections', 'corpora', 'databases', 'benchmarks']},'IS_LOWER': False, "IS_PUNCT": False, 
                                  'POS':{'IN': ['PROPN', 'NOUN']}}, 
                           {'TEXT': {'NOT_IN': [',','.','[', ']']},'IS_LOWER': False, 'OP':'*'}]
                    matcher.add('Dataset', None, pattern11)
                    doc2 = ' '.join([str(token) for token in doc[multiple[0]:multiple[1]]])
                    doc3 = nlp(doc2)
                    matches2 = matcher(doc3)
                    for match_id, start, end in matches2:
                        try: 
                            store[end + multiple[0]]
                        except:
                            store[end + multiple[0]] = start + multiple[0]
                        if store[end + multiple[0]] > start + multiple[0]:
                            store[end + multiple[0]] = start + multiple[0]
                    for key in store.keys():
                        try:
                            store2[store[key]]
                        except:
                            store2[store[key]] = key
                        if store2[store[key]] < key:
                            store2[store[key]] = key
    sent = nlp(sentence)
    sent = [str(token) for token in sent]
    tags = nltk.pos_tag(sent)
    end = 0
    for i in range(len(sent)):
        if any([t.isalpha() for t in str(sent[i])]) or any([t.isdigit for t in str(sent[i])]) or any([t.ispunct for t in str(sent[i])]):
            if len(store2) == 0:
                dataset = dataset.append({'Sentence #':'Sentence: ' + str(number), 'Word':tags[i][0], 'POS': tags[i][1], 'Tag': 'O'}, ignore_index=True)
            elif i not in store2.keys() and i >= end:
                dataset = dataset.append({'Sentence #':'Sentence: ' + str(number), 'Word':tags[i][0], 'POS': tags[i][1], 'Tag': 'O'}, ignore_index=True)
            elif i in store2.keys():
                dataset = dataset.append({'Sentence #':'Sentence: ' + str(number), 'Word':tags[i][0], 'POS': tags[i][1], 'Tag': 'B'}, ignore_index=True)
                end = store2[i]
            else:
                dataset = dataset.append({'Sentence #':'Sentence: ' + str(number), 'Word':tags[i][0], 'POS': tags[i][1], 'Tag': 'I'}, ignore_index=True)

# Delete # to see how far the rule-based system is 
#     print(x)
    x+=1
    



In [ ]:
# Save the dataset, changee name to give the file an other name
dataset.to_csv('RULEBASED.csv')